# Webscraping Traditional NBA Pipeline

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup as bs
from time import sleep
import requests

import pandas as pd
import numpy as np

In [2]:
# Make a url list
url_base = 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season='
url_list = []
a = 2021

while a >= 1996:
    b = a+1
    new_url = url_base + str(a) + "-" + str(b)[-2:] + "&SeasonType=Regular+Season"
    a -=1
    url_list.append(new_url)

In [3]:
url_list[:5]

['https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2021-22&SeasonType=Regular+Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2020-21&SeasonType=Regular+Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2019-20&SeasonType=Regular+Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2018-19&SeasonType=Regular+Season',
 'https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1&Season=2017-18&SeasonType=Regular+Season']

In [8]:
dataframe_list = []
n=1 # Make a counter to see how many webpages are scraped

# Scrape stat table for each url in url_list
for url in url_list:
    # Use Selenium, provide path to the Edge driver
    s=Service("C:\\Users\sandr\OneDrive\Desktop\Drivers\msedgedriver.exe")
    driver = webdriver.Edge(service=s)
    
    # Make python pause for some time
    sleep(2)

    # Get the page HTML
    try:
        driver.get(url)
        # Make python pause for some time
        sleep(3)
    except WebDriverException:
        print("Webdriver is acting immediately after ‘closing’ the browser")

    # Use Selenium, click on accept cookies button
    WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='I Accept']"))).click()

    # Use Selenium, select table page
    select_page = Select(driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
    select_page.select_by_visible_text('All')

    # Make python pause for some time
    sleep(3)
    
    # Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
    nba_soup = bs(driver.page_source, 'lxml')
    
    ########## To view all tables on webpage ##########
#     for table in nba_soup.find_all('table'):
#         print(table.get('class'))
    
    # Creating list with all tables
    tables = nba_soup.find_all('table')
    
    # Locate the appropriate table on the page
    table = nba_soup.find('table', class_='Crom_table__p1iZz')
   
    # Find all the headers in the columns
    headers=[]
    for i in table.find_all('th'):
        title = i.text.replace("\n", "").strip()
        headers.append(title)
    
    # Remove redundant headers
    headers_2 = ['PLAYER','TEAM','AGE','GP','W','L','MIN','PTS','FGM','FGA',
                 'FG%','3PM','3PA','3P%','FTM','FTA','FT%','OREB','DREB', 
                 'REB','AST','TOV','STL','BLK','PF','FP','DD2','TD3','+/-','SEASON']

    # Create a dataframe using the headers as column names
    nba_stat = pd.DataFrame(columns = headers_2) 

    # Create a for loop to fill nba_stat
    for i in table.find_all('tr')[1:]:
        row_data = i.find_all('td')[1:]
        row =[j.text.replace("\n", "") for j in row_data] 
        
        # get the season year
        get_season = url.split('Season=')[1] 
        # get the season year
        get_season = get_season.split('&SeasonType')[0] 
        # add the season to the row column
        row.append(get_season)
        length = len(nba_stat)
        nba_stat.loc[length] = row
    
    print("Completed:", n, "webpage(s) scraped.")
    n +=1 # add 1 to counter every successful page retrieved
        
    # Close all browsers
    driver.quit()

    # Append table to list of dataframes
    dataframe_list.append(nba_stat)

Completed: 1 webpage(s) scraped.
Completed: 2 webpage(s) scraped.
Completed: 3 webpage(s) scraped.
Completed: 4 webpage(s) scraped.
Completed: 5 webpage(s) scraped.
Completed: 6 webpage(s) scraped.
Completed: 7 webpage(s) scraped.
Completed: 8 webpage(s) scraped.
Completed: 9 webpage(s) scraped.
Completed: 10 webpage(s) scraped.
Completed: 11 webpage(s) scraped.
Completed: 12 webpage(s) scraped.
Completed: 13 webpage(s) scraped.
Completed: 14 webpage(s) scraped.
Completed: 15 webpage(s) scraped.
Completed: 16 webpage(s) scraped.
Completed: 17 webpage(s) scraped.
Completed: 18 webpage(s) scraped.
Completed: 19 webpage(s) scraped.
Completed: 20 webpage(s) scraped.
Completed: 21 webpage(s) scraped.
Completed: 22 webpage(s) scraped.
Completed: 23 webpage(s) scraped.
Completed: 24 webpage(s) scraped.
Completed: 25 webpage(s) scraped.
Completed: 26 webpage(s) scraped.


In [9]:
# dataframe_list is a list of all dataframes
dataframe_list

[             PLAYER TEAM AGE  GP   W   L     MIN   PTS  FGM   FGA  ...  AST  \
 0        Trae Young  ATL  23  76  40  36  2651.9  2155  711  1544  ...  737   
 1     DeMar DeRozan  CHI  32  76  43  33  2742.9  2118  774  1535  ...  374   
 2       Joel Embiid  PHI  28  68  45  23  2296.4  2079  666  1334  ...  284   
 3      Jayson Tatum  BOS  24  76  49  27  2731.0  2046  708  1564  ...  334   
 4      Nikola Jokic  DEN  27  74  46  28  2475.6  2004  764  1311  ...  584   
 ..              ...  ...  ..  ..  ..  ..     ...   ...  ...   ...  ...  ...   
 600     Nate Hinton  IND  23   2   0   2     2.4     0    0     1  ...    0   
 601      Sam Dekker  TOR  28   1   1   0     0.9     0    0     0  ...    0   
 602  Trayvon Palmer  DET  27   1   0   1    16.8     0    0     1  ...    0   
 603      Tyler Hall  NYK  25   1   1   0     1.9     0    0     0  ...    0   
 604  Zylan Cheatham  UTA  26   1   0   1     5.2     0    0     3  ...    0   
 
      TOV  STL BLK   PF    FP DD2 TD3 

In [10]:
# Concat all the dataframes into one table
nba_stats = pd.concat(dataframe_list, ignore_index=True)
nba_stats.head()

,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,SEASON
0,Trae Young,ATL,23,76,40,36,2651.9,2155,711,1544,...,737,303,72,7,128,3535,42,0,159,2021-22
1,DeMar DeRozan,CHI,32,76,43,33,2742.9,2118,774,1535,...,374,181,68,24,178,3244,6,0,77,2021-22
2,Joel Embiid,PHI,28,68,45,23,2296.4,2079,666,1334,...,284,214,77,99,181,3774,46,2,368,2021-22
3,Jayson Tatum,BOS,24,76,49,27,2731.0,2046,708,1564,...,334,217,75,49,174,3433,22,0,667,2021-22
4,Nikola Jokic,DEN,27,74,46,28,2475.6,2004,764,1311,...,584,281,109,63,191,4338,66,19,444,2021-22


# Save File

In [11]:
# Save as csv file
nba_stats.to_csv('nba_stats_raw.csv', encoding='utf-8', index=False)